In [ ]:
!pip install -e ../

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
from pathlib import Path
import logging

import src

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

In [ ]:
config_name = "config_v3.yaml"

cfg = src.utility.run_debug(
    lambda: src.utility.load_config(config_name)
)
# cfg["gpu"] = 0
# cfg["train"]["val_batch_size"] = 1

dataset, _ = src.dataset.get_train_val_dataset(cfg,
                                               data_dir=src.constant.DATA_ROOT / "train_images",
                                               csv_fname=src.constant.DATA_ROOT / "train.csv",
                                               is_train=False)
result_dir = src.constant.OUTPUT_ROOT / cfg.title
model_weights = ["best_val_acc0.hdf5"]
models = [src.predict.get_and_load_model(cfg, result_dir / model_name) for model_name in model_weights]
preds, labels = src.predict.predict(dataset=dataset, models=models)

## XGBoostお試し

In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
param_dist = {
    'num_class': cfg.n_classes,
    'max_depth': 10,
    'objective':'multi:softmax',
    'n_estimators': 1000,
}

clf = xgb.XGBClassifier(**param_dist)

X_train, X_test, y_train, y_test = train_test_split(preds, labels.argmax(axis=1), test_size=0.2)

clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric='mlogloss',
        early_stopping_rounds=5,
        verbose=True)

evals_result = clf.evals_result()